# Correlation Between News and Stock Movement

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
import os
import pandas as pd

# Load the datasets from the specified folders
def load_data():
    stock_data_folder = '../data/yfinance_data/'  # Folder for stock data
    analyst_ratings_file = '../data/raw_analyst_ratings.csv'  # Path for analyst ratings
    
    stock_data = {}
    
    try:
        # Load each historical data file into a dictionary
        for file_name in os.listdir(stock_data_folder):
            if file_name.endswith('.csv'):  # Check if the file is a CSV
                ticker = file_name.split('_')[0]  # Extract ticker symbol
                stock_data[ticker] = pd.read_csv(os.path.join(stock_data_folder, file_name))
        
        # Load the analyst ratings (which includes news data)
        news_df = pd.read_csv(analyst_ratings_file)
        
        return stock_data, news_df
    except Exception as e:
        print(f"Error loading datasets: {e}")

# Load datasets
stock_data, news_df = load_data()

### Date Alignment

Convert date columns in the news DataFrame to datetime format and normalize dates to ensure alignment.

In [ ]:
try:
    # Convert the 'date' column in news_df to datetime
    news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
    if news_df['date'].isnull().any():
        print("Warning: Some dates could not be parsed and are set to NaT.")

    for ticker in stock_data:
        stock_data[ticker]['Date'] = pd.to_datetime(stock_data[ticker]['Date'], errors='coerce')
        stock_data[ticker] = stock_data[ticker].set_index('Date').resample('D').ffill().reset_index()

except Exception as e:
    print(f"Error converting date columns: {e}")


In [28]:
print(news_df[news_df['date'].isnull()])

         Unnamed: 0                                           headline  \
10               10    30 Stocks Moving in Friday's Pre-Market Session   
11               11  SVB Leerink Maintains Outperform on Agilent Te...   
12               12  8 Stocks Moving In Thursday's After-Hours Session   
13               13  Agilent Technologies shares are trading higher...   
14               14  Agilent Technologies Q2 Adj. EPS $0.71 Beats $...   
...             ...                                                ...   
1407323     1413844             Top Narrow Based Indexes For August 29   
1407324     1413845  Recap: Wednesday's Top Percentage Gainers and ...   
1407325     1413846  UPDATE: Oppenheimer Color on China Zenix Auto ...   
1407326     1413847  Oppenheimer Initiates China Zenix At Outperfor...   
1407327     1413848  China Zenix Auto International Opens For Tradi...   

                                                       url  \
10       https://www.benzinga.com/news/20/05/1609

In [29]:
# Remove rows where the 'date' could not be parsed
news_df = news_df.dropna(subset=['date'])


In [30]:
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')


In [37]:
print(news_df['date'].head(20))


0      2020-06-05 10:30:54-04:00
1      2020-06-03 10:45:20-04:00
2      2020-05-26 04:30:07-04:00
3      2020-05-22 12:45:06-04:00
4      2020-05-22 11:38:59-04:00
5      2020-05-22 11:23:25-04:00
6      2020-05-22 09:36:20-04:00
7      2020-05-22 09:07:04-04:00
8      2020-05-22 08:37:59-04:00
9      2020-05-22 08:06:17-04:00
1433   2020-06-09 10:52:15-04:00
1434   2020-06-08 11:29:29-04:00
1435   2020-06-08 10:32:42-04:00
1436   2020-06-05 07:40:08-04:00
1437   2020-06-04 14:46:13-04:00
1438   2020-06-03 10:19:06-04:00
1439   2020-05-27 10:10:20-04:00
1440   2020-05-27 07:32:46-04:00
1441   2020-05-26 07:47:48-04:00
1442   2020-05-18 10:50:11-04:00
Name: date, dtype: datetime64[ns, UTC-04:00]


In [33]:
try:
    for ticker in stock_data:
        stock_data[ticker]['Date'] = pd.to_datetime(stock_data[ticker]['Date'], errors='coerce')
        stock_data[ticker] = stock_data[ticker].dropna(subset=['Date'])
        stock_data[ticker] = stock_data[ticker].set_index('Date').resample('D').ffill().reset_index()
except Exception as e:
    print(f"Error converting date columns: {e}")


In [38]:

try:
    for ticker in stock_data:
        print(f"Columns in {ticker} before conversion: {stock_data[ticker].columns.tolist()}")
        
        print(f"\nSample 'Date' values before conversion for {ticker}:")
        print(stock_data[ticker]['Date'].head())
        
        stock_data[ticker]['Date'] = pd.to_datetime(stock_data[ticker]['Date'], errors='coerce')
        
        print(f"\nSample 'Date' values after conversion for {ticker}:")
        print(stock_data[ticker]['Date'].head())
        
        num_invalid_dates = stock_data[ticker]['Date'].isnull().sum()
        print(f"{ticker} has {num_invalid_dates} invalid dates (NaT) after conversion.")
        
        stock_data[ticker] = stock_data[ticker].dropna(subset=['Date'])
        print(f"{ticker} shape after dropping NaT: {stock_data[ticker].shape}")
        
        stock_data[ticker] = stock_data[ticker].set_index('Date').resample('D').ffill().reset_index()
        print(f"\nSample data for {ticker} after resampling:")
        print(stock_data[ticker].head())
        
except Exception as e:
    print(f"Error converting date columns: {e}")


Columns in AAPL before conversion: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits']

Sample 'Date' values before conversion for AAPL:
0   1980-12-12
1   1980-12-13
2   1980-12-14
3   1980-12-15
4   1980-12-16
Name: Date, dtype: datetime64[ns]

Sample 'Date' values after conversion for AAPL:
0   1980-12-12
1   1980-12-13
2   1980-12-14
3   1980-12-15
4   1980-12-16
Name: Date, dtype: datetime64[ns]
AAPL has 0 invalid dates (NaT) after conversion.
AAPL shape after dropping NaT: (15937, 9)

Sample data for AAPL after resampling:
        Date      Open      High       Low     Close  Adj Close     Volume  \
0 1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1 1980-12-13  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
2 1980-12-14  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
3 1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   
4 1980-12-16  0.113281  0.113281  

### Sentiment Analysis

Define a function to analyze the sentiment of news headlines and apply it to create a new column with scores.

In [39]:
from textblob import TextBlob

def analyze_sentiment(headline):
    """Analyze sentiment of a headline and return the polarity score."""
    return TextBlob(headline).sentiment.polarity

# Apply sentiment analysis to news headlines
try:
    news_df['sentiment'] = news_df['headline'].apply(analyze_sentiment)
except Exception as e:
    print(f"Error during sentiment analysis: {e}")

### Calculate Daily Stock Returns

Compute the daily percentage change in stock prices to represent movements.

In [40]:
try:
    for ticker in stock_data:
        stock_data[ticker]['daily_return'] = stock_data[ticker]['Close'].pct_change()  # Calculate daily returns
except Exception as e:
    print(f"Error calculating daily returns: {e}")

### Correlation Analysis

Aggregate daily sentiment scores by averaging them and merge sentiment data with stock data to calculate the Pearson correlation coefficient.

In [43]:
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
news_df = news_df.dropna(subset=['date'])
news_df.rename(columns={'date': 'Date'}, inplace=True)


In [44]:
for ticker in stock_data:
    stock_data[ticker]['Date'] = pd.to_datetime(stock_data[ticker]['Date'], errors='coerce')
    stock_data[ticker] = stock_data[ticker].dropna(subset=['Date'])
    stock_data[ticker] = stock_data[ticker].set_index('Date').resample('D').ffill().reset_index()


In [45]:
daily_sentiment = news_df.groupby('Date')['sentiment'].mean().reset_index()


In [48]:
# For news_df / daily_sentiment
daily_sentiment['Date'] = daily_sentiment['Date'].dt.tz_localize(None)

# For stock_data (example for AAPL)
stock_data['AAPL']['Date'] = stock_data['AAPL']['Date'].dt.tz_localize(None)



In [49]:
try:
    combined_df = pd.merge(stock_data['AAPL'], daily_sentiment, on='Date', how='inner')
    correlation = combined_df['sentiment'].corr(combined_df['daily_return'])
    print(f"Pearson correlation coefficient for AAPL: {correlation}")
except Exception as e:
    print(f"Error calculating correlation: {e}")


Pearson correlation coefficient for AAPL: nan


c:\Users\metya\anaconda3\envs\financial_analysis\Lib\site-packages\numpy\lib\_function_base_impl.py:3037: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\metya\anaconda3\envs\financial_analysis\Lib\site-packages\numpy\lib\_function_base_impl.py:2894: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\metya\anaconda3\envs\financial_analysis\Lib\site-packages\numpy\lib\_function_base_impl.py:2894: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
